<b>`LONDON SMART METER DATA ANALYSIS`</b>

There are `5` important files in this dataset :

<b>`informations_households.csv`</b> : This file that contains all the information on the households in the panel (their acorn group, their tariff) and in which block.csv.gz file their data are stored.

<b>`halfhourly_dataset` </b>: These files that contain the block files with the half-hourly smart meter measurement

<b>`daily_dataset`</b>: These files that contain the block files with the daily information like the number of measures, minimum, maximum, mean, median, sum and std.

<b>`acorn_details.csv`</b> : Details on the acorn groups and their profile of the people in the group.

<b>`weatherdailydarksky.csv`</b> : This contains the daily data from darksky api. 

Our analysis makes use of all these files to bring out insights

In [0]:
# Importing all required libraries
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# Importing all blocks from Daily_Dataset Folder into single dataframe

daily_df_final = pd.DataFrame()

for i in range(0,111):
    path = 'https://raw.githubusercontent.com/OmkarKatkade/Data-603/main/daily_dataset/daily_dataset/block_' + str(i) + '.csv';
    daily_df = pd.read_csv(path)
    daily_df_final = daily_df_final.append(daily_df)
    
daily_df_final
df = spark.createDataFrame(daily_df_final)

In [0]:
#Importing Household data

household = pd.read_csv("https://raw.githubusercontent.com/OmkarKatkade/Data-603/main/informations_households.csv", encoding= 'unicode_escape')
household = spark.createDataFrame(household)
household.show()

+---------+--------+-------+-------------+-------+
|    LCLid|stdorToU|  Acorn|Acorn_grouped|   file|
+---------+--------+-------+-------------+-------+
|MAC005492|     ToU| ACORN-|       ACORN-|block_0|
|MAC001074|     ToU| ACORN-|       ACORN-|block_0|
|MAC000002|     Std|ACORN-A|     Affluent|block_0|
|MAC003613|     Std|ACORN-A|     Affluent|block_0|
|MAC003597|     Std|ACORN-A|     Affluent|block_0|
|MAC003579|     Std|ACORN-A|     Affluent|block_0|
|MAC003566|     Std|ACORN-A|     Affluent|block_0|
|MAC003557|     Std|ACORN-A|     Affluent|block_0|
|MAC003553|     Std|ACORN-A|     Affluent|block_0|
|MAC003482|     Std|ACORN-A|     Affluent|block_0|
|MAC003463|     Std|ACORN-A|     Affluent|block_0|
|MAC003449|     Std|ACORN-A|     Affluent|block_0|
|MAC003428|     Std|ACORN-A|     Affluent|block_0|
|MAC003423|     Std|ACORN-A|     Affluent|block_0|
|MAC003422|     Std|ACORN-A|     Affluent|block_0|
|MAC003400|     Std|ACORN-A|     Affluent|block_0|
|MAC003394|     Std|ACORN-A|   

In [0]:
# Merging household data with daily dataset using inner join on 'LCLid'
df = df.join(household, ['LCLid'], how='inner')
df.show()

+---------+----------+------------------+------------------+----------+------------+------------------+------------------+------------------+--------+-------+-------------+-------+
|    LCLid|       day|     energy_median|       energy_mean|energy_max|energy_count|        energy_std|        energy_sum|        energy_min|stdorToU|  Acorn|Acorn_grouped|   file|
+---------+----------+------------------+------------------+----------+------------+------------------+------------------+------------------+--------+-------+-------------+-------+
|MAC004029|2012-07-06|            0.4625|0.5432916666666666|      1.17|          24|0.3022701756011846|            13.039|             0.096|     Std|ACORN-C|     Affluent|block_5|
|MAC004029|2012-07-07|            0.2255|          0.253125|     0.696|          48|0.1583076220258815|             12.15|             0.049|     Std|ACORN-C|     Affluent|block_5|
|MAC004029|2012-07-08|0.1654999999999999|0.2285416666666666|      0.84|          48|0.182216840

<b>Data Cleaning</b>

In [0]:
#Dropping the 'energy_std' column as it is less relevant to our analysis
df = df.drop('energy_std')

In [0]:
#Dropping rows with nulls values
df = df.dropna()

In [0]:
#Lets see the count of rows based on Acorn Group
count1 = df.groupBy('Acorn_grouped').count().orderBy('count')
display(count1)

Acorn_grouped,count
ACORN-,781
ACORN-U,19644
Comfortable,937061
Adversity,1135331
Affluent,1407678


INFERENCE: Group "ACORN-" and "ACORN-U" are significantly lower in terms of no. of records. Therefore we can ignore these two groups for our analysis.

In [0]:
# Dropping 'ACORN-' and 'ACORN-U' groups from the dataset.
df = df.filter(df.Acorn_grouped != 'ACORN-')
df = df.filter(df.Acorn_grouped != 'ACORN-U')

<b>Exploratory Data Analysis</b>

<b> `1. YEAR WISE ENERGY CONSUMPTION TREND BASED ON ACORN GROUP `</b>

In [0]:
'''
Extracting Day of the week, Month and Year from the 'day' column.
This will help in aggregation going forward
'''

df = df.withColumn('dayOfWeek', dayofweek(col('day')))
df = df.withColumn('month', month(col('day')))
df = df.withColumn('year', year(col('day')))
df.printSchema()

root
 |-- LCLid: string (nullable = true)
 |-- day: string (nullable = true)
 |-- energy_median: double (nullable = true)
 |-- energy_mean: double (nullable = true)
 |-- energy_max: double (nullable = true)
 |-- energy_count: long (nullable = true)
 |-- energy_sum: double (nullable = true)
 |-- energy_min: double (nullable = true)
 |-- stdorToU: string (nullable = true)
 |-- Acorn: string (nullable = true)
 |-- Acorn_grouped: string (nullable = true)
 |-- file: string (nullable = true)
 |-- dayOfWeek: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)



In [0]:
'''
Now that we have Day of the week, Month and Year
extracted from the 'day' column, we can go ahead and check
year-wise energy consumption based on acorn group.
'''

t1 = df.groupBy("year", "Acorn_grouped").sum("energy_sum").withColumnRenamed("sum(energy_sum)", "Total_Energy_Consumption(kWh)").collect()
display(t1)

year,Acorn_grouped,Total_Energy_Consumption(kWh)
2012,Affluent,5905451.254876445
2014,Affluent,1496936.1979974064
2011,Affluent,46217.2389995
2013,Affluent,8709586.250781491
2014,Comfortable,904786.9739640976
2012,Comfortable,3109564.0009666
2011,Comfortable,28142.78900109998
2013,Comfortable,5332295.709879235
2011,Adversity,28592.354996600014
2012,Adversity,3353160.413988029


<b>`INFERENCE:` </b> Since we have significantly lower number of records for the year 2011 and 2014 and therefore energy consumption for these years is shown significantly lower than other years in the dataset. It is also important to notice that the households belonging to the Acorn Group 'Affluent' tend to consume the most electricity. Our initial assumption was that the group 'Comfortable' would be consumping a lot more electricity than the group 'Adversity' but it turns out that both these groups consume almost equal amount of electricity.

<b> `2. MONTHLY ENERGY CONSUMPTION BASED ON ACORN GROUP `</b>

In [0]:
'''
Now that we have seen the yearly trend of energy consumption,
we are curious to see the monthly trend of energy consumption across 
these acorn groups. 
'''
t2 = df.groupBy("month", "Acorn_grouped").sum("energy_sum").withColumnRenamed("sum(energy_sum)", "Total_Energy_Consumption(kWh)").collect()

In [0]:
display(t2)

month,Acorn_grouped,Total_Energy_Consumption(kWh)
2,Affluent,1705632.3189626068
1,Affluent,1900032.3069310065
4,Affluent,998308.6029894009
11,Affluent,1649664.247970792
10,Affluent,1436340.5169611035
8,Affluent,1084161.1579873997
6,Affluent,1059878.7469981017
3,Affluent,1136309.5819712994
5,Affluent,1047072.5709592995
9,Affluent,1197749.6989761982


<b>`INFERENCE:` </b> It is quite evident from the above visualization that households from the Affluent group tend to consume maximum electricity irrespective of the month. Additionally, it is also important to notice that the month of <b>January</b> is when the energy consumption is at the peak across all acorn groups. The Affluent group consumped around <b> 1.9 million kWh </b> of energy in the month of January across all years in the dataset.

<b> `3. DAILY ENERGY CONSUMPTION`</b>

In [0]:
'''
Let's see the trend of the total energy consumption on daily basis
'''

# energy_sum = df.select("day", "energy_sum").where(df.Acorn_grouped == 'Affluent')
energy_sum_0 = df.groupBy('day', 'LCLid').avg('energy_sum').withColumnRenamed("avg(energy_sum)", "Avg_Energy_Consumption(kWh)").collect()
display(energy_sum_0)

day,LCLid,Avg_Energy_Consumption(kWh)
2012-07-06,MAC004029,13.039
2012-07-07,MAC004029,12.15
2012-07-08,MAC004029,10.97
2012-07-09,MAC004029,13.679000000000002
2012-07-10,MAC004029,20.203
2012-07-11,MAC004029,12.423
2012-07-12,MAC004029,13.437
2012-07-13,MAC004029,20.385
2012-07-14,MAC004029,11.707
2012-07-15,MAC004029,17.380000000000003


<b>`INFERENCE:` </b> The total energy consumption reaches the peak in January 2013 with the value of 81.2 thousand kWh.

<b> `4. How much energy on an average does a typical household in UK consume in a day across the year. `</b>

In [0]:
'''
It is also important to know the average of how much energy does a typical household in UK consume in a day across the year.
'''
energy_sum_0 = spark.createDataFrame(energy_sum_0)
energy_sum_0 = energy_sum_0.groupBy('day').avg('Avg_Energy_Consumption(kWh)').collect()
display(energy_sum_0)

day,avg(Avg_Energy_Consumption(kWh))
2013-03-14,11.75087685989583
2014-02-16,11.580864293667071
2014-02-22,10.771510681257562
2012-10-21,10.942398086705422
2012-03-04,13.475282051165502
2013-08-20,7.657049635396703
2013-12-28,11.37412870699881
2012-03-09,12.162531287156973
2012-01-12,12.062221444755243
2013-11-08,10.527465251352927


<b>`INFERENCE:` </b> The average energy consumption for any household in a day in UK ranges from 0-16 kWh. The maximum average energy in a day is 15.9 kWh and the least is 7.22 kWh.

<b> `5. How much energy on an average does a typical household in UK consume in a day across the year based on the ACORN group. `</b>

In [0]:
'''
Let us see how this trend is distributed among the Acorn groups
'''
gbda = df.groupBy(['day', 'Acorn_grouped']).mean('energy_sum')
display(gbda)

day,Acorn_grouped,avg(energy_sum)
2013-06-20,Affluent,9.03160066905787
2013-09-24,Affluent,9.526662889058652
2014-02-27,Affluent,12.113709676430807
2012-07-17,Affluent,8.985623008366542
2012-07-27,Affluent,8.924739433565389
2013-02-14,Affluent,14.118078346824293
2013-11-04,Affluent,11.992550000495045
2013-12-11,Affluent,13.729320524317496
2014-02-05,Affluent,13.408866392446043
2012-06-20,Affluent,8.979145150946643


<b>`INFERENCE:` </b> It is quite evident from the above visualization that Affluent group tops the group in terms of daily average energy consumption.

In [0]:
df = df.withColumn("day", to_date("day", 'yyyy-MM-dd'))

In [0]:
#Importing all blocks from half_hourly_dataset

hh_df_final = pd.DataFrame()

for i in range(0, 50):
    path = 'https://raw.githubusercontent.com/OmkarKatkade/Data-603/main/halfhourly_dataset/halfhourly_dataset/block_' + str(i) +'.csv';
    hh_df = pd.read_csv(path)
    hh_df_final = hh_df_final.append(hh_df)
    
hh_df_final

The python kernel is unresponsive.

In [0]:
'''
Here we are performing Data Cleaning and Data Manipulation for the Half Hourly Dataset.
Replacing all null values in the column 'energy(kWh)' with 0. There aren't many such instances though.
'''
hh_df_final["energy(kWh/hh)"] = hh_df_final["energy(kWh/hh)"].fillna(0)
hh_df_final["energy(kWh/hh)"] = hh_df_final["energy(kWh/hh)"].replace("Null", 0)
hh_df_final["energy(kWh/hh)"] = pd.to_numeric(hh_df_final["energy(kWh/hh)"])

'''
Converting the data type of the column 'date' from string to date.
'''
hh_df_final["date"] = pd.to_datetime(hh_df_final['tstp']).dt.date

'''
Creating a dataframe
'''
hh_df = spark.createDataFrame(hh_df_final)
hh_df.printSchema()

'''
Converting the column 'tstp' from string type to timestamp and
also extracting the hour from the timestamp in a different column
'''
hh_df = hh_df.withColumn("datetype_timestamp", to_timestamp(col("tstp")))
hh_df = hh_df.withColumn("hour", hour(col("tstp")))


In [0]:
hh_df.show(10)

<b> ` 6. Which hours of the day consumes the most and the least energy?`</b>

In [0]:
'''
Since we have information on the half hourly energy consumption,
we are curious to know which hours of the day consumes the most energy.
'''
dfrt = hh_df.groupBy("hour").sum("energy(kWh/hh)").withColumnRenamed('sum(energy(kWh/hh))', 'Energy_Sum')
display(dfrt)

<b>`INFERENCE:` </b> The energy consumption increases drastically from 15th hour (3 p.m) and reaches the peak at 19th hour (7 p.m).

In [0]:
'''
Extracting date from the datetime in the half hourly dataset.
'''
hh_df_final["date"] = pd.to_datetime(hh_df_final['tstp']).dt.date

In [0]:
#Importing weather daily data
weather = pd.read_csv("https://raw.githubusercontent.com/OmkarKatkade/Data-603/main/weather_daily_darksky.csv")
weather["date"] = pd.to_datetime(weather['temperatureMaxTime'])
weather_df = spark.createDataFrame(weather)
weather_df.show()

In [0]:
'''
Extracting only the important features from the weather dataset
'''
weather_df_1 = weather_df.select("date","temperatureMax", "humidity")
wtr = weather_df_1.withColumn("day",to_date("date")).collect()
weather_df_1 = spark.createDataFrame(wtr)
weather_df_1.show()

In [0]:
'''
Joining the important features from the weather dataset with the daily dataset based on the day
'''
df_new = df.join(weather_df_1, ['day'], how='inner')
df_new.show()

In [0]:
'''
Here the aim is to analyze the impact of Temperature and Humidity on the energy consumption.
'''

df_grped_by_day = df_new.select("day", "temperatureMax", "month", "energy_sum").groupBy("day").sum("energy_sum")
df_grped_for_hum = df_new.select("day","humidity","month","energy_sum").groupBy("day").sum("energy_sum")

df_grped_by_day1 = df_new.select("day", "temperatureMax").groupBy("day").mean("temperatureMax")
df_grped_for_hum1 = df_new.select("day", "humidity").groupBy("day").mean("humidity")

df_grped_by_day = df_grped_by_day.join(df_grped_by_day1, ["day"])
df_grped_for_hum = df_grped_for_hum.join(df_grped_for_hum1, ["day"])

df_grped_by_day = df_grped_by_day.withColumn('dayOfWeek', dayofweek(col('day')))
df_grped_by_day = df_grped_by_day.withColumn('month', month(col('day')))
df_grped_by_day = df_grped_by_day.withColumn('year', year(col('day')))

df_grped_for_hum = df_grped_for_hum.withColumn('dayOfWeek', dayofweek(col('day')))
df_grped_for_hum = df_grped_for_hum.withColumn('month', month(col('day')))
df_grped_for_hum = df_grped_for_hum.withColumn('year', year(col('day')))

df_temp_temp = df_grped_by_day.groupBy("month").avg("avg(temperatureMax)").withColumnRenamed("avg(avg(temperatureMax))", "Temperature")
df_temp2_temp = df_grped_by_day.groupBy("month").avg("sum(energy_sum)").withColumnRenamed("avg(sum(energy_sum))", "Avg_Energy_Sum")
df_temp3_temp = df_temp_temp.join(df_temp2_temp, ["month"], how="inner")

df_temp = df_grped_for_hum.groupBy("month").avg("avg(humidity)").withColumnRenamed("avg(avg(humidity))", "Humidity")
df_temp2 = df_grped_for_hum.groupBy("month").avg("sum(energy_sum)").withColumnRenamed("avg(sum(energy_sum))", "Avg_Energy_Sum")
df_temp3 = df_temp.join(df_temp2, ["month"], how="inner")

pandasDF = df_temp3_temp.toPandas()
pandasDF1 = df_temp3.toPandas()

<b> ` 7. What is the impact of Humidity on the energy consumption?`</b>

In [0]:
'''
Visualizing the impact of humidity on the energy consumption
'''
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.lineplot(x = pandasDF1['month'], y = pandasDF1['Humidity'], legend= True);
ax2 = plt.twinx()
sns.lineplot(x = pandasDF1['month'], y = pandasDF1['Avg_Energy_Sum'], color="g", ax=ax2, legend = True)
plt.legend(labels=["Avg_Energy_Sum"])
plt.show()


<b>`INFERENCE:` </b> From the above visualization, we can see that energy consumption is directly proportional to the humidity for the household in the UK.

<b> ` 8. What is the impact of Temperature on the energy consumption?`</b>

In [0]:
'''
Visualizing the impact of temperature on the energy consumption
'''
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax = sns.lineplot(x = pandasDF['month'], y = pandasDF['Temperature'], legend= 'auto');
ax2 = plt.twinx()
sns.lineplot(x = pandasDF['month'], y = pandasDF['Avg_Energy_Sum'], color="g", ax=ax2, legend = 'auto')
plt.legend(labels=["Avg_Energy_Sum"])
plt.show()

<b>`INFERENCE:` </b> From the visualization above, we can say that as the energy consumption increases with the decrease in temperature.